In [74]:
import os
import pickle
import glob
import pandas as pd
import xml.etree.ElementTree as ET

# Construct empty DataFrame
df_premises = pd.DataFrame(columns=[
    'document', 'file_path', 'split', 'text', 'type'
])

def parse_xml(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()

    documents = []

    for elem in root.iter():
        if elem.tag == "OP" or elem.tag == "reply":
            documents.append((elem.text, "None"))
            all_descendants = [b for b in elem.iter() if b is not elem]
            for desc in all_descendants:
                documents.append((desc.text, desc.tag))
    return documents

# Retrieve premises from all XML files
for split in ['positive', 'negative']:
    for path in glob.glob(os.path.join(f'v2.0/{split}', '*.xml')):
        try:
            for premise, argument in parse_xml(path):
                df_premises = pd.concat([df_premises, pd.DataFrame({
                    'document': [path.split('/')[2].split('.')[0]],
                    'file_path': [path],
                    'split': [path.split('/')[1]],
                    'text': [premise],
                    'type': [argument],
                })], axis=0, ignore_index=True)
        except ET.ParseError:
            pass

#df_premises.to_pickle('./v2.0-processed/df_premises.pickle')

In [75]:
df_premises

,document,file_path,split,text,type
0,88,v2.0/positive/88.xml,positive,\n\n,None
1,88,v2.0/positive/88.xml,positive,[47% of all jobs are at risk of being automate...,premise
2,88,v2.0/positive/88.xml,positive,This number will grow grow until the vast majo...,premise
3,88,v2.0/positive/88.xml,positive,Since some of this automation will inevitably ...,premise
4,88,v2.0/positive/88.xml,positive,By fortifying themselves in their gated commun...,premise
...,...,...,...,...,...
4097,85,v2.0/negative/85.xml,negative,They couldn't just survive together,claim
4098,85,v2.0/negative/85.xml,negative,the ecosystem will not be able to sustain the ...,premise
4099,85,v2.0/negative/85.xml,negative,The rural folk can survive without the urban folk,claim
4100,85,v2.0/negative/85.xml,negative,their standard of living will plummet due to l...,claim
